# Start
these libraries are neccesary to install to use this code. specific versions must be installed. just run the code below for the first time when you will run this file.

## Important variables to set before running code 


In [1]:
# set those thresholds based on your needs
blurryness_threshold = 50
reflection_threshold = 0.7
yolo_confidence_score_threshold = 0.8
confidence_score_classification_threshold = 0.9
classification_threshold = 0.95

In [2]:
# !pip install pillow==10.4.0
# !pip install tensorflow==2.8.3
# !pip install ultralytics==8.3.23
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install opencv-python==4.10.0.84
# !pip install numpy==1.26.4
# !pip install pytesseract==0.3.13


In [3]:
import os
import time
from datetime import datetime
import json
import random
import tkinter as tk
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img,img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input, decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import RandomCrop, RandomFlip, RandomRotation, RandomContrast
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.keras import backend as K
import torch
from ultralytics import YOLO
from tkinter import simpledialog, filedialog
import pytesseract 
from deepface import DeepFace

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# Clear last session to feee up space before new
K.clear_session()

# if GPU is available this code will state 1
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# image height and width for input into Siamese Network with MobileNetV3 as a base 
IMG_HEIGHT = 300
IMG_WIDTH = 300
# important for tesseract to work in jupyter notebook
# PATH is relative chooose the one where tesseract.exe is installed
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe' 


# Options for the dropdown menu
languages = {"Georgian":"kat", "French": "fra", "German": "deu", "Spanish" :"spa", "Russian":"rus"}

Num GPUs Available:  1


# AI code part for object detection and id card recognition

In [4]:
# load Siamese network model file
def load_model(model_name):
    model = tf.keras.models.load_model(model_name)
    return model

# Preprocessing function to resize and normalize image
def preprocess_image(image):
    # resize image to match height and width specified earlier 
    image_resized = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    
    # Normalize values of pixels to be around 0.0-1.0
    image_resized = image_resized / 255.0  
    
    return image_resized

## Id cad type prediction function

In [5]:
# siamese network prediction function  
def predict(id_images, img_to_evaluate, model, threshold = classification_threshold):
    '''
    id_images - batch of saved id images, should be numpy array of shape (x, 300, 300, 3) where x is number of saved images 
    or path for image which is saved in the memory and should be compared to, in this case it is string path
    img_to_evaluate - image which will be compared to img_base which is saved in the memory
    model - AI model file which contains trained weights for siamese network
    threshold - optional value which we can state for siamese networks confidence score from 0.50 till 0.99, which means 
    that results will be shown only if confidence score for similarity is more than threshold
    '''
    
    # preprocess images
    if type(id_images) == str:
        img_base = np.array(preprocess_image(id_images))
        img_base = np.expand_dims(img_base, axis=0)  # Add batch dimension, shape becomes (1, 300, 300, 3)
    elif isinstance(id_images, np.ndarray):
        img_base = id_images

    
    img_to_evaluate = np.array(preprocess_image(img_to_evaluate))

    # expand dimensions of images to add batch dimension 
    
    img_to_evaluate = np.expand_dims(img_to_evaluate, axis=0)  # Add batch dimension, shape becomes (1, 300, 300, 3)
    
    img_to_evaluate = np.tile(img_to_evaluate, (len(img_base), 1, 1, 1))
    
    # predict the similarity between the two images,
    # returns tensor with two values like this [0.37, 0.63] which states how similar are images
    # 0.0 no similarity , 1.0 full similarity
    preds = model.predict((img_base,img_to_evaluate))
    predicted_idx = np.argmax(preds[:, 1])

    if preds[predicted_idx][1] > threshold:
        return predicted_idx, preds[predicted_idx][1]
    else:
        return 999,999
    # if the model is sure that shown images are more similar than threshold it will output the 1, if not 0
    # if predictions are less than a threshold it will output 2   
    

### Load models for Object detection and recognition

In [6]:
# load the siamese network model
model_name = f'MobileNetV3_1024_siamese.h5'
model = load_model(model_name)
# load the YOLO model 
yolo_model = torch.hub.load('ultralytics/yolov5',"custom", path=r'.\yolov5\runs\train\exp6\weights/best.pt',force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\Saba/.cache\torch\hub\master.zip
YOLOv5  2025-1-5 Python-3.9.21 torch-2.5.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


## YOLO object detection functions

In [7]:
# function returns YOLO model object detection results 
def yolo(yolo_model,frame):
    results = yolo_model(frame)
    return results

kk = cv2.imread("./id_examples/driving_license_geo_20241219_102226.jpg")
kk = cv2.resize(kk,(300,300))
print(kk.shape)
kkk = yolo(yolo_model,kk)
predict(np.array([kk]),kk,model)
# unpack the coordinates of bounding boxes and confidence score for ID class
def get_results_of_yolo(frame_predictions):
    x1 = int(frame_predictions.xyxy[0][0][0])
    y1 = int(frame_predictions.xyxy[0][0][1])
    x2 = int(frame_predictions.xyxy[0][0][2])
    y2 = int(frame_predictions.xyxy[0][0][3])
    
    confidence = frame_predictions.xyxy[0][0][4]
    
    return x1,y1,x2,y2,confidence

# crop the id card from the frame for it to be saved or passed to siamese network for evaluating
def crop_id_card(frame_predictions, frame):
    x1,y1,x2,y2,confidence = get_results_of_yolo(frame_predictions)
    # x1 (pixels)  y1 (pixels)  x2 (pixels)  y2 (pixels)   confidence   class
    cropped_image = frame[y1:y2, x1:x2]
    
    return cropped_image

(300, 300, 3)


### Id card recognition function
we need this function to retrieve already saved ids

In [8]:
# file path for id_examples
id_examples_filepath = './id_examples/'
def retrieve_saved_ids():
    saved_ids_labels = []
    saved_ids_images = []
    for id_img in os.listdir(id_examples_filepath):
        id_image_uncropped = cv2.imread(id_examples_filepath+id_img)
        saved_ids_images.append(id_image_uncropped)
        saved_ids_labels.append(id_img)

    id_img_batch = np.array([preprocess_image(image_array) for image_array in saved_ids_images])
    if id_img_batch.size == 0:
        id_img_batch = np.ones((1, 300, 300, 3), dtype=np.uint8) * 255
        saved_ids_labels.append("Nothing")
    return id_img_batch, saved_ids_labels



# GUI

In [9]:
def add_new_id_to_database(frame, new_id_name):
    # Create a directory to store new ID images if it doesn’t exist
    os.makedirs("./id_examples/", exist_ok=True)

    # Create a unique file name with ID type and current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path = f"./id_examples/{new_id_name}_{timestamp}.jpg"

    # Save the image to the specified path
    cv2.imwrite(file_path, frame)


In [10]:
def ask_user_for_id_type():
    root = tk.Tk()
    root.withdraw()
    new_id_type = simpledialog.askstring("Input", "Enter new ID type name:")
    root.destroy()
    return new_id_type
    
# Create a button that prints the input field's text when clicked
def on_button_click(id_image):
    new_id_name = input_entry.get()  # Get text from entry field
    print(new_id_name + "es daarqva")
    if new_id_name != None:
        add_new_id_to_database(id_image, new_id_name)
        id_images_batch, id_labels = retrieve_saved_ids()
        input_entry.delete(0, tk.END)  # Clear the entry field after clicking
    else:
        return None

def create_window_submit_window():
    window_submit_photo = tk.TK()
    window_submit_photo.title("ID submission")
    window.geometry("1200x800")
    
    video_label_submit = tk.Label(window_submit_photo)
    video_label_submit.pack()
    
    input_entry = tk.Entry(window, width=30)
    input_entry.pack()

    button = tk.Button(window, text="Submit", command=on_button_click)
    button.pack()
    
def check_for_new_id_type(frame,confidence_score):
    global frame_counter
    if confidence_score > confidence_score_threshold:
        if frame_counter < 20:
            frame_counter += 1
            return False
        else:
            new_id_name = ask_user_for_id_type()
            if new_id_name != None:
                add_new_id_to_database(frame, new_id_name)
                frame_counter = 0
                return True
            else:
                frame_counter = 0
                return False

    frame_counter = 0
    return False

def check_id_page2(image,siamese_conf_score):
    if siamese_conf_score > classification_threshold:
        new_id_name = ask_user_for_id_type()
        if new_id_name != None:
            add_new_id_to_database(image, new_id_name)
            
            
            return True
        else:
            return False

# Detecting, segmenting and recognizing descriptive regions in ID cards
Part of code for subproject part 2

## Functions checking blurryness and reflections

In [11]:
def is_blurry(image, threshold=blurryness_threshold):
    """
    Check if an image is blurry using the variance of the Laplacian.
    
    Parameters:
    image (numpy.ndarray): The input image.
    threshold (float): Threshold below which the image is considered blurry.
    
    Returns:
    bool: True if the image is blurry, False otherwise.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return laplacian_var < threshold, laplacian_var

def has_reflection(image, bright_threshold=200, reflection_ratio=reflection_threshold):
    """
    Check if an image has significant reflections based on bright regions.
    
    Parameters:
    image (numpy.ndarray): The input image.
    bright_threshold (int): Pixel intensity threshold to consider as bright.
    reflection_ratio (float): Proportion of bright pixels to consider as reflection.
    
    Returns:
    bool: True if the image has high reflection, False otherwise.
    float reflection_percentage
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    bright_pixels = cv2.threshold(gray, bright_threshold, 255, cv2.THRESH_BINARY)[1]
    bright_pixel_count = np.sum(bright_pixels == 255)
    total_pixels = gray.size
    reflection_percentage = bright_pixel_count / total_pixels
    return reflection_percentage > reflection_ratio, reflection_percentage



## Function for extracting faces


In [12]:
def find_face(image):
    height, width, dim = image.shape
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        if 5 < ((w*h) * 100)/(height * width) < 50:
            image = image[y:y + h, x:x + w]
            return image
    return None
    

## Sharpening images with convolutional kernel

In [13]:
def sharpen_with_kernel(image):
    """
    Sharpen an image using a convolutional kernel.
    
    Parameters:
    image (numpy.ndarray): The input image.
    
    Returns:
    numpy.ndarray: The sharpened image.
    """
    # Define a sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

    # Apply the kernel to the image
    sharpened = cv2.filter2D(image, -1, kernel)
    
    return sharpened

## signature detection

In [14]:

def find_signature(image):
    '''
    finds signature based on contours
    
    Parameters:
    image (numpy.ndarray): The input image.
    
    Returns:
    numpy.ndarray: The sharpened image.
    
    '''
    
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 47))

    # Apply blackhat morphology
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
    
    # Threshold and find contours
    _, thresh = cv2.threshold(blackhat, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    

    # Filter contours for signature area
    height, width, dim = image.shape
    for c in contours:
        x, y, w, h = cv2.boundingRect(c)
        if 1.5 < ((w*h) * 100)/(height * width) < 8:
            aspect_ratio = w / float(h)
    
            if 1.5 < aspect_ratio < 6.5: 
                
                # image = image[y:y + h, x:x + w]
                image = image[y:y + h, x:x + w]
                return image
    return None

## Function for detecting machine readable zone (MRZ) on the back of id

In [55]:
def find_mrz_area(image):
    """
    Find the area where the MRZ is located in an ID card image.
    
    Parameters:
    image (numpy.ndarray): The input ID card image.
    
    Returns:
    tuple: The bounding box (x, y, w, h) of the MRZ area.
    """

    # initialize a rectangular and square structuring kernel (this size is dependent on the ID-Card size)
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 5))
    sqKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 20))

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce noise and improve contour detection
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    # Apply blackhat morphological operation to enhance the text
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKernel)
    
    # apply a closing operation using the rectangular kernel to close
	# gaps in between letters
    blackhat_closed = cv2.morphologyEx(blackhat, cv2.MORPH_CLOSE, rectKernel)
    
    # then apply Otsu's thresholding method
    thresh = cv2.threshold(blackhat_closed, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # perform another closing operation, this time using the square
	# kernel to close gaps between lines of the MRZ
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel)

    # Convert the thresholded image to a color image to plot the contours in color
    thresh_color = cv2.cvtColor(thresh, cv2.COLOR_GRAY2BGR)
  
	# find contours in the thresholded image and sort them by their size
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
    
    # Handle different versions of OpenCV
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Sort contours by area in descending order    
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    # Initialize ROI as None
    roi = None

	# loop over the contours
    for i, c in enumerate(cnts):
		# compute the bounding box of the contour and use the contour to
		# compute the aspect ratio and coverage ratio of the bounding box
		# width to the width of the image
        (x, y, w, h) = cv2.boundingRect(c)
         # Draw the bounding box on the color thresholded image
        color = (0, 255, 0) if i == 0 else (0, 0, 255)  # Green for the first (largest) contour, red for others
        cv2.rectangle(thresh_color, (x, y), (x + w, y + h), color, 2)
        # cv2.imshow('Thresholded Image with Contours', thresh_color)

        ar = w / float(h)
        crWidth = w / float(gray.shape[1])
		# check to see if the aspect ratio and coverage width are within
		# acceptable criteria
        if ar > 4 and crWidth > 0.25:
            print("aris")
			# pad the bounding box to have some space for later reading
            pad = 5
            x = x - pad
            y = y - pad
            w = w + (pad * 2)
            h = h + (pad * 2)

			# extract the ROI from the image and draw a bounding box
			# surrounding the MRZ
            roi = image[y:y + h, x:x + w]

            break
    
    return (x, y, w, h), roi if roi is not None else None

## Exctracting text information from id using tesseract

In [56]:
# def extract_text(image,lang="eng",config = "--psm 7 outputbase digits"):
#     text = pytesseract.image_to_string(image, lang=lang,conf)
#     return text

In [57]:
def check_validity(cropped_id_card):
    blurry, sharpness = is_blurry(cropped_id_card,blurryness_threshold)
    reflection, refl_score = has_reflection(cropped_id_card)
    print(f'sharpness: {sharpness}, reflection score: {refl_score:.2%}', end='\r' )
    if not blurry and not reflection:
        return True, sharpness, refl_score
    else:
        return False, sharpness, refl_score

In [58]:

chosen_language = "eng"
is_running = True

# Open video capture (0 is the default camera)
# video_path = "./passport_video.mp4"
cap = cv2.VideoCapture(0)

frame_counter = 0  # Use local variable instead of global

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0:  # Avoid division by zero
    fps = 30  # Default to 30 FPS if unable to retrieve
frame_delay = int(1000 / fps) # Calculate frame delay in milliseconds

print(frame_delay)



# Initialize the Tkinter window
window = tk.Toplevel()

window.title("ID Detection")
window.geometry("1280x860")

# Configure rows and columns to allow frames to expand
window.grid_rowconfigure(0, weight=1)  # Make row 0 expandable
window.grid_columnconfigure(0, weight=1) 

def show_frame(frame):
    frame.tkraise()  # Bring the frame to the top
    
def init_text_label(page,text,row,column):
    label = tk.Label(page, text=text, font=("Arial", 12), fg="#1c1c1e")
    label.grid(row=row, column=column, padx=5, pady=5, sticky="w") 
    return label
# Function to handle the selection
def on_select(option):
    global chosen_language
    chosen_language = languages[option]
    label_lang.config(text=f"Choosen Language: {chosen_language}")
    
def init_dropdown_menu(selected_language,row,column):
     # Create a dropdown menu
    dropdown = tk.OptionMenu(page2, selected_language, *languages.keys(), command=on_select)
    dropdown.config(width=15, font=("Arial", 12))
    dropdown.grid(row=row, column=column, padx=5, pady=5, sticky="w") 
    return dropdown

def init_image_label(page,row,column):
    # Create a label to display the image
    image_label = tk.Label(page)
    image_label.grid(row=row, column=column, padx=10, pady=5, sticky="w") 
    return image_label

def show_image(image,image_label):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(image_rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    image_label.imgtk = imgtk
    image_label.config(image=imgtk)

def face_image_resize(face_image):
    if face_image is not None:
        if face_image.shape[0]>0 and face_image.shape[1]>0:
            face_image = cv2.resize(face_image, (200,150))
            return face_image

def signature_image_resize(signature_image):
    if signature_image is not None:
        if signature_image.shape[0]>0 and signature_image.shape[1]>0:
            # signature_image = cv2.resize(signature_image,(int(signature_image.shape[0] * 1.5), int(signature_image.shape[1] * 1.5)))
            return signature_image
def show_page_2(image):
    global label_lang
    page2.tkraise()
    
    label_prediction = init_text_label(page2, "prediction for id: ", 0, 2)
    
    label_lang_text = f"Choosen Language: {chosen_language}"
    label_lang = init_text_label(page2, label_lang_text, 1, 2)
    
    # Tkinter variable to store the selected option
    selected_language = tk.StringVar(value="Select Language")
    
     # Create a dropdown menu
    dropdown = init_dropdown_menu(selected_language,2,2)
    
    id_image_label = init_image_label(page2,0,0)
    show_image(image,id_image_label)

    face_image = find_face(image)
    if face_image is not None:
        face_image = face_image_resize(face_image)
        face_image_label = init_image_label(page2,2,0)
        show_image(face_image,face_image_label)

    signature_image = find_signature(image)
    if signature_image is not None:
        signature_image_label = init_image_label(page2,3,0)
        signature_image = signature_image_resize(signature_image)
        show_image(signature_image,signature_image_label)
        
    mrz_area, roi = find_mrz_area(image)
    if roi is not None:
        if roi.shape[0]>0 and roi.shape[1]>0:
            mrz_image_label = init_image_label(page2, 4, 0)
            show_image(roi,mrz_image_label)
            
    

    prediction_name, confidence_score_siamese = prediction_for_label(image,model)
    if prediction_name != "not found":
        print(prediction_name)
        label_prediction.config(text= f'id type: {prediction_name}, {confidence_score_siamese:.2f}')
    else:
        prediction_name, confidence_score_siamese = prediction_for_label(image,model)
        label_prediction.config(text= f'id type: {prediction_name}, {confidence_score_siamese:.2f}')
        
    window.update()
   
def prediction_for_label(sharpened_cropped_id, model):
    id_images_batch, id_labels = retrieve_saved_ids()
    # # Predict with siamese network
    prediction_idx, confidence_score_siamese = predict(id_images_batch, sharpened_cropped_id, model)
    
    if prediction_idx == 999:
        prediction_name = "not found"
        if check_id_page2(sharpened_cropped_id, confidence_score_siamese) == True:
            id_images_batch, id_labels = retrieve_saved_ids()
    else:
        prediction_name = id_labels[prediction_idx]
    return prediction_name, confidence_score_siamese
    

# Create two frames for different pages
page1 = tk.Frame(window, bg="lightblue")
page2 = tk.Frame(window, bg="lightgreen")
page3 = tk.Frame(window)

for frame in (page1, page2, page3):
    frame.grid(row=0, column=0, sticky="nsew")  # Sticky ensures the frame fills the window

# Create a label to display the video feed
video_label = init_image_label(page1,0,0)


frame_size_x = 1.2
video_frame_size = (int(400 * frame_size_x), int(300 * frame_size_x))
def on_capture_id_click(cur_sharp, sharp_hist, cur_refl, refl_hist,image):
    global is_running
    mean_refl = calculate_mean(refl_hist)
    mean_sharp = calculate_mean(sharp_hist)
    if cur_sharp < mean_sharp:
        label_warning = tk.Label(page1, text=f"image is blurry, please stand still", font=("Arial", 12))
        label_warning.grid(row=1,column=1, padx=10, pady=5, sticky="w") 
    elif cur_refl < mean_refl:
        label_warning = tk.Label(page1, text=f"Id card has reflections, please get rid of them", font=("Arial", 12))
        label_warning.grid(row=1,column=1, padx=10, pady=5, sticky="w") 
    else:
        is_running = False
        cap.release()
        cv2.destroyAllWindows()# Release the camera
        show_page_2(image)
        
        

def calculate_mean(hist_list):
    return int(sum(hist_list)/len(hist_list))
               
button_capture_id = tk.Button(page1, text="Click Me", font=("Arial", 12), bg="orange")
button_capture_id.grid(row=2, column=0, padx=10, pady=10, sticky="w")  # Align to the left


label_info = init_text_label(page1,"", 3, 1)

# label_text = tk.Label(window, text="text from passport", font=("Arial", 10), fg="#1c1c1e")
# label_text.grid(row=0, column=1, padx=10, pady=5) 

sharpness_hist = []
reflection_score_hist = []
start_time = time.time()


# Function to update frame in Tkinter window
show_frame(page1)
def update_frame():
    global frame_counter, id_images_batch, id_labels,is_running

    # Capture frame-by-frame
    ret, frame = cap.read()
    if not is_running:
        return  
    
    if not ret:
        print("Failed to grab frame.")
        window.after(10, update_frame)  # Schedule next frame update
        return
    
   
    frame = cv2.resize(frame, video_frame_size)
    
    # Predict bounding box coordinates
    bounding_box_prediction = yolo(yolo_model, frame)
    
    # Make siamese network work only if ID card is found
    if bounding_box_prediction.xyxy[0].shape != (0, 6):
        # Get coordinates of bounding box
        
        x1, y1, x2, y2, confidence_score_object_det = get_results_of_yolo(bounding_box_prediction)
        
        # Crop the ID card
        cropped_id_card = crop_id_card(bounding_box_prediction, frame)
        is_valid_for_anaysis, sharpness, refl_score = check_validity(cropped_id_card)
        sharpness_hist.append(sharpness)
        reflection_score_hist.append(refl_score)
        if is_valid_for_anaysis:
            sharpened_cropped_id = sharpen_with_kernel(cropped_id_card)
            if confidence_score_object_det > yolo_confidence_score_threshold:
                # Draw bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                text = f'{confidence_score_object_det:.2f}'
                text_position = (x1, y1 - 10)
                cv2.putText(frame, text, text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                button_capture_id.config(command=lambda: on_capture_id_click(sharpness,sharpness_hist,refl_score,reflection_score_hist,sharpened_cropped_id))
            
        label_info.config(text=f'sharpness: {sharpness}(mean: {calculate_mean(sharpness_hist)}), \n reflection score: {refl_score:.2%}')
        
            
                
                        
            
                
        #     if face_image is not None:
        #         result = DeepFace.verify(
        #                   img1_path = face_image,
        #                   img2_path = "img2.jpg",
        #                 )
                    
        #     extracted_text = extract_text(sharpened_cropped_id, lang=chosen_language)
        #     label_text.config(text=extracted_text)
        #     window.update()
        
        # # Update the GUI
        # window.update()  # Processes pending events
    show_image(frame, video_label)
    # Convert frame to RGB and update the Tkinter label
    # frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # img = Image.fromarray(frame_rgb)
    # imgtk = ImageTk.PhotoImage(image=img)
    # video_label.imgtk = imgtk
    # video_label.configure(image=imgtk)

    
    
    # Schedule the next frame update
    video_label.after(frame_delay, update_frame)


    
        

# Start updating frames
update_frame()

# Define a function to handle window close
def on_closing():
    cap.release()
    cv2.destroyAllWindows()# Release the camera
    window.destroy()  # Close the Tkinter window
    
# Bind the window close event
window.protocol("WM_DELETE_WINDOW", on_closing)

# Start the Tkinter main loop
window.mainloop()


33
german_20241230_010738.jpg53, reflection score: 14.27%%


In [19]:
calculate_mean(sharpness_hist)

168

In [190]:
import re

def extract_text(image, lang=chosen_language):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.fastNlMeansDenoising(image, None, 18, 2, 31)
    
    cv2.imshow("bef", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    text = pytesseract.image_to_string(image, lang='spa+eng')
    return text


# def extract_name(text):
#     name = re.search(r"PRIMER APELLIDO:\s*(.*)", text).group(1)
    

# surname = re.search(r"Surname:\s*(.*)", text).group(1)
# dob = re.search(r"DOB:\s*(.*)", text).group(1)
image = cv2.imread("./id_examples/spanish_id_1.jpg")
print(image.shape)
# Crop region of interest (e.g., for name)
image = image[30:280, 150:400]

image = cv2.resize(image, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)


extracted_text = extract_text(image)
extracted_text
# cv2.imshow("img", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

(380, 600, 3)


'PRIMER APELLIDO\n\nESPAÑOL\n\nSEGUNDO APELLIDO\nESPAÑOL\nNOMBRE\n\nJUAN\n\nSEXO NACIONALIDAD\n\nM ESP\n\nFECHA DE NACIMIENTO\n\n1950\nIDESP\nAAA023112\n\nVALIDO HASTA\n\n17 04 2018\nNO aa\n'

In [201]:

def extract_surname(text,second_surname=False):
    
    if second_surname == False:
        # Define a regex to match the label and extract the value
        pattern = r"PRIMER APELLIDO\s*\n+([^\n]+(?:\n-s)?)"
    else:
        pattern = r"SEGUNDO APELLIDO\s*\n+([^\n]+(?:\n-s)?)"
    # Search for the pattern
    match = re.search(pattern, text, re.IGNORECASE)

    if match:
        extracted_name = match.group(1).strip()  # Extract the value and clean it
        return extracted_name
    else:
        return None


def extract_name(text):
    pattern = r"nNOMBRE\s*\n+([^\n]+(?:\n-s)?)"

'ESPAÑOL'

In [186]:


def detect_text_fields(image):
    image = cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # gray = cv2.fastNlMeansDenoising(gray, None, 28, 1, 31)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    cv2.imshow("blur", blurred)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Use adaptive thresholding to detect edges
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY_INV, 25, 25)  

    cv2.imshow("thresh", thresh)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Morphological operations to close gaps
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 3))
    morphed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)


    # find contours in the thresholded image and sort them by their size
    cnts = cv2.findContours(morphed.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
    
    # Handle different versions of OpenCV
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Sort contours by area in descending order    
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    # Draw bounding boxes around detected text fields
    # loop over the contours
    for i, c in enumerate(cnts):
		# compute the bounding box of the contour and use the contour to
		# compute the aspect ratio and coverage ratio of the bounding box
		# width to the width of the image
        (x, y, w, h) = cv2.boundingRect(c)
         # Draw the bounding box on the color thresholded image
        # ar = w / float(h)
        crWidth = w / float(gray.shape[1])
        if crWidth > 0.05:
            color = (0, 255, 0) if i == 0 else (0, 0, 255)  # Green for the first (largest) contour, red for others
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        # cv2.imshow('Thresholded Image with Contours', thresh_color)

  #       ar = w / float(h)
  #       crWidth = w / float(gray.shape[1])
		# # check to see if the aspect ratio and coverage width are within
		# # acceptable criteria
  #       if ar > 2 and crWidth > 0.5:
  #           # print("aris")
		# 	# pad the bounding box to have some space for later reading
  #           pad = 5
  #           x = x - pad
  #           y = y - pad
  #           w = w + (pad * 2)
  #           h = h + (pad * 2)

		# 	# extract the ROI from the image and draw a bounding box
		# 	# surrounding the MRZ
  #           roi = image[y:y + h, x:x + w]

            
    

    # Save the output image
    cv2.imshow("fields", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [180]:
image = cv2.imread("./id_examples/spanish_id_1.jpg")
print(image.shape)
# Crop region of interest (e.g., for name)
image = image[30:280, 150:400]

image = cv2.resize(image, None, fx=2.5, fy=2.5, interpolation=cv2.INTER_CUBIC)


extracted_text = extract_text(image)
print(extracted_text)
cv2.imshow("img", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(380, 600, 3)
PRIMER APELLIDO

ESPAÑOL

SEGUNDO APELLIDO
ESPAÑOL
NOMBRE

JUAN

SEXO NACIONALIDAD

M ESP

FECHA DE NACIMIENTO

1950
IDESP
AAA023112

VALIDO HASTA

17 04 2018
NO aa



In [170]:
detect_text_fields(image)

In [20]:
cv2.destroyAllWindows()

In [2]:
def kk():
    return b + 5 
def kkk():
    global b
    b = 5
    c = kk()
    return a + c
a = 15

kkk()

25